In [1]:
import pandas as pd
import seaborn as sns
import numpy as np


In [2]:
df=pd.read_csv('/content/cleaned_dataset.csv')

In [3]:
df = df.drop(columns=['car'])

In [4]:
df.columns

Index(['destination', 'passanger', 'weather', 'temperature', 'time', 'coupon',
       'expiration', 'gender', 'age', 'maritalStatus', 'has_children',
       'education', 'occupation', 'income', 'Bar', 'CoffeeHouse', 'CarryAway',
       'RestaurantLessThan20', 'Restaurant20To50', 'toCoupon_GEQ5min',
       'toCoupon_GEQ15min', 'toCoupon_GEQ25min', 'direction_same',
       'direction_opp', 'Y'],
      dtype='object')

In [5]:
from sklearn.preprocessing import LabelEncoder
ordinal_cols = ['income', 'education', 'occupation']
for cols in ordinal_cols:
    le = LabelEncoder()
    df[cols] = le.fit_transform(df[cols])
df = pd.get_dummies(df, drop_first=True)



In [6]:
df.shape

(5829, 57)

In [7]:
from sklearn.model_selection import train_test_split


X = df.drop(columns=["Y"])  #
y = df["Y"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier(n_estimators=100, random_state=42)

rf.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [9]:
from sklearn.metrics import accuracy_score, classification_report


y_pred = rf.predict(X_test)


print("Accuracy:", accuracy_score(y_test, y_pred)*100)


print(classification_report(y_test, y_pred))
#so my model is giving 77% accuracy
#s

Accuracy: 77.35849056603774
              precision    recall  f1-score   support

         0.0       0.76      0.71      0.73       510
         1.0       0.78      0.82      0.80       656

    accuracy                           0.77      1166
   macro avg       0.77      0.77      0.77      1166
weighted avg       0.77      0.77      0.77      1166



In [10]:
#now we will tune the hyper parameters for this to improve accuracy

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

param_dist = {
    "n_estimators": np.random.randint(50, 500, 10),
    "max_depth": np.append(np.random.randint(5, 50, 5), None),
    "min_samples_split": np.random.randint(2, 20, 5),
    "min_samples_leaf": np.random.randint(1, 10, 5),
    "max_features": ["sqrt", "log2", None],
    "bootstrap": [True, False],
    "criterion": ["gini", "entropy"]
}

Grid Search CV

In [20]:
from sklearn.model_selection import GridSearchCV

rf_grid=GridSearchCV(estimator=rf,param_grid=param_dist,cv=2,n_jobs=-1)

In [21]:
rf_grid.fit(X_train,y_train)# tried running it but it was taklin too much time so randomised search is better hnece i terminated the process

KeyboardInterrupt: 

In [23]:
from sklearn.model_selection import RandomizedSearchCV
rf_grid=RandomizedSearchCV(estimator=rf,param_distributions=param_dist,cv=2,n_jobs=-1)

In [24]:
rf_grid.fit(X_train,y_train)

RandomizedSearchCV(cv=2, estimator=RandomForestClassifier(random_state=42),
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'criterion': ['gini', 'entropy'],
                                        'max_depth': array([36, 10, 29, 42, 20, None], dtype=object),
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_leaf': array([9, 6, 7, 4, 5]),
                                        'min_samples_split': array([19,  3, 19,  3,  6]),
                                        'n_estimators': array([233, 397, 448, 101, 210, 411, 119, 190, 446, 285])})

In [25]:
rf_grid.best_params_

{'n_estimators': np.int64(411),
 'min_samples_split': np.int64(19),
 'min_samples_leaf': np.int64(6),
 'max_features': 'sqrt',
 'max_depth': 20,
 'criterion': 'gini',
 'bootstrap': False}

In [26]:
rf_grid.best_score_ #best score is 73% earlier it was 77% because it was random grid search took >30mins

np.float64(0.7325746170085793)